In [ ]:
# Constants and modules
import ipywidgets as widgets
from IPython.display import display
import my_apk as apkPY

PARAMETERS: dict[str, set] = {
    "": set(),
    "build": {"net", "positionnal_args", "dir"},
    "decode": {"no_res", "no_dis", "positionnal_args", "file"},
    "patch": {"arch", "gadget-conf", "net", "positionnal_args", "file"},
    "rename": {"net", "file", "pkg_name"},
    "everything": {"net", "positionnal_args", "dir", "no_res", "no_dis", "arch", "gadget-conf", "file", "pkg_name"}
}

for _,_v in PARAMETERS.items():
    _v.add("label_mandatory")
    _v.add("label_optional")
    _v.add("run")

PARAMETERS[""] = set()

STYLE_TITLE = {"description_width": "initial", "font_weight": "bold"}

In [ ]:
# Logic and widgets for notebook
class choiceWidget(widgets.VBox):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.action = widgets.Dropdown(
            options=PARAMETERS.keys(),
            description='Action:')
        self.action.observe(on_action_change, names='value')

        self.label_mandatory = widgets.Label(
            value='Mandatory arguments:', style=STYLE_TITLE)
        # label_mandatory.style.font_weight = 'bold'

        self.file = widgets.Text(
            description='File:',
            placeholder='your_file.apk',
            value='',
        )

        self.dir = widgets.Text(
            description='Folder:',
            placeholder='your_folder\\',
            value='',
        )

        self.apkname = widgets.Text(
            description='Apk name:',
            placeholder='com.example.app',
            value='',
        )

        self.arch = widgets.Dropdown(
            options=apkPY.SUPPORTED_ARCH,
            value=apkPY.SUPPORTED_ARCH[0],
            description='Arch:',
            disabled=False,
            indent=True
        )

        self.label_optional = widgets.Label(
            value='\nOptional arguments:', style=STYLE_TITLE)

        self.net = widgets.Checkbox(
            value=False,
            description='Net',
            disabled=False,
            indent=True
        )

        self.no_res = widgets.Checkbox(
            value=False,
            description='No ressources',
            disabled=False,
            indent=True
        )

        self.no_dis = widgets.Checkbox(
            value=False,
            description='Don\'t decompile',
            disabled=False,
            indent=True
        )

        self.gadget_conf = widgets.Text(
            value='',
            placeholder='config.json',
            description='Gadget config:',
            style={
                "description_width": "initial"
            }
        )

        # self.positional = widgets.Text(
        #     value='',
        #     placeholder='--example...',
        #     description='Positional args:',
        #     disabled=False,
        #     style={
        #         "description_width": "initial"
        #     }
        # )
        self.positional = widgets.TagsInput(
            value=[],
            allow_duplicates=True,
            style={
                "description_width": "initial"
            },
            placeholder='--example...',
        )
        self.positional_box = widgets.HBox(
            [widgets.Label(value='Positional args:'), self.positional])

        self.run = widgets.Button(
            description='Run!',
            disabled=False,
            button_style='success',
            tooltip='Run',
            icon='terminal',
        )
        # Center the text of the button bc vbox breaks it
        self.run_box = widgets.Box([self.run])
        self.run.on_click(on_run)

        self.widgets_: dict[str, widgets.DOMWidget] = {
            'label_mandatory': self.label_mandatory,
            'file': self.file, 'dir': self.dir, 'pkg_name': self.apkname,
            'arch': self.arch, 'label_optional': self.label_optional,
            'net': self.net, 'no_res': self.no_res,
            'no_dis': self.no_dis, 'gadget-conf': self.gadget_conf,
            'positionnal_args': self.positional_box, 'run': self.run_box
        }

        for w in self.widgets_.values():
            w.layout.display = "none"


def on_action_change(change):
    with output:
        show_params(change['new'])


def on_run(b):
    output.clear_output()
    with output:
        print("Running...")
        print("Command: ", end='')

        # Collect all the values
        to_show = {"action": choice_widget.action.value}
        for widget_name in PARAMETERS[choice_widget.action.value]:
            widget = choice_widget.widgets_[widget_name]
            if isinstance(widget, widgets.HBox):
                widget = widget.children[1]
            elif not isinstance(widget, (widgets.Text, widgets.Checkbox,
                                         widgets.Dropdown, widgets.TagsInput)):
                continue
            if widget.value is False or widget.value == '' or widget.value == []:
                continue
            to_show[widget_name] = widget.value

        parser, args = apkPY.get_parser(to_show)
        parser(**args)

        print("Done!")


def show_params(action):
    """Logic to decide which parameters to show/hide
    """
    global current

    full = PARAMETERS[action]

    to_add = full - current
    to_remove = current - full
    current = full

    w_to_add = [choice_widget.widgets_[name] for name in to_add]
    w_to_remove = [choice_widget.widgets_[name] for name in to_remove]

    for w in w_to_add:
        w.layout.display = "inherit"

    for w in w_to_remove:
        w.layout.display = "none"

choice_widget = choiceWidget()
output = widgets.Output()
current = set()


In [ ]:
# For when debugging
try:
    import pydevd # noqa
    debugger = getattr(getattr(pydevd.GetGlobalDebugger(
    ), "_on_configuration_done_event", None), "_flag", False)
except ImportError:
    debugger = False
    pass
if debugger:
    on_run("")


In [ ]:
# Show the widgets
output = widgets.Output()
display(choice_widget.action)
display(widgets.VBox(list(choice_widget.widgets_.values())), output )
